In [31]:
import os
import sys
import requests
import re
import time
import json
import pandas as pd
import csv
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

In [32]:
extracts = re.compile('[^ 가-힣|a-z|A-Z|0-9|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')
cost_extracts = re.compile('[^ 가-힣|a-z|A-Z|\[|\]|(|)|-|~|?|!|.|,|:|;|%]+')

In [33]:
data_url = open('./data/goodchoice.csv', 'w', encoding='utf8') # url 저장 관련 csv파일
data = open('./data/info.csv', 'w', encoding='utf8') # 정보 저장관련 csv 파일

In [34]:
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window() #전체 화면
Url = 'https://www.goodchoice.kr/product/search/1/7052' #서울 역삼, 강남, 논현
driver.get(Url)

In [37]:
urls = driver.find_element(By.ID, "poduct_list_area")
urlList = urls.find_elements(By.TAG_NAME, 'li')

for li in urlList:
    aTag = li.find_element(By.TAG_NAME, 'a')
    href = aTag.get_attribute('href')
    data_url.write(href)
    data_url.write('\n')

data_url.close()
    

In [39]:
data.write('"basic_info"')
data.write(', ')
data.write('"cost_info"')
data.write('\n')

1

In [40]:
f = open('C:/Users/xmssn/OneDrive/바탕 화면/대학/3학년1학기/빅데이터시스템/project/python_01/data/goodchoice.csv', 'r', encoding='utf8')
rdr = csv.reader(f)

In [41]:
# 기본 정보 및 요금 정보
for line in rdr:
    addr = line[0]
    rs = requests.get(addr)
    rs.encoding = None
    html = rs.text
    soup = bs(html, 'html.parser')

    tarbasic_info = soup.select('#content > article.detail_info > section.default_info')
    tarcost_info = soup.select('#content > article.detail_info.on > section.table_wrap > div:nth-child(2) > table:nth-child(2)')

    basic_info = ''
    for text in tarbasic_info:
        basic_info += text.getText()

    basic_info = extracts.sub('', basic_info)
    basic_info = re.sub(' +', ' ', basic_info)
    basic_info = '"'+basic_info+'"'

    rcost_info = soup.find("table", {"class" : "table_type_02"})
    cost_info = rcost_info.find("span")
    cost_info = cost_info.text

    data.write(basic_info)
    data.write(', ')
    data.write(cost_info)
    data.write('\n')

f.close()
data.close()

KeyboardInterrupt: 